In [1]:
import kagglehub
import pandas as pd
from dotenv import load_dotenv
from pprint import pprint
import os

c:\Users\karen\miniconda3\envs\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get Dog Breeds Details from Kaggle

In [2]:

# Download latest version
path = kagglehub.dataset_download("warcoder/dog-breeds-details")

# Load the dataset
df = pd.read_csv(path + "/dog_breeds.csv")
df['Name'] = df['Name'].str.lower()
df.head()

,Name,min_life_expectancy,max_life_expectancy,max_height_male,max_height_female,max_weight_male,max_weight_female,min_height_male,min_height_female,min_weight_male,...,shedding,grooming,drooling,coat_length,good_with_strangers,playfulness,protectiveness,trainability,energy,barking
0,golden retriever,10,12,24.0,24.0,75.0,65.0,23.0,23.0,65.0,...,4,2,2,1,5,4,3,5,3,1
1,dachshund,12,16,9.0,9.0,32.0,32.0,8.0,8.0,16.0,...,2,2,2,2,4,4,4,4,3,5
2,labrador retriever,10,12,24.5,24.5,80.0,70.0,22.5,22.5,65.0,...,4,2,2,1,5,5,3,5,5,3
3,great dane,7,10,32.0,32.0,175.0,140.0,30.0,30.0,140.0,...,3,1,4,1,3,4,5,3,4,3
4,boxer,10,12,25.0,25.0,80.0,65.0,23.0,23.0,65.0,...,2,2,3,1,4,4,4,4,4,3


Get Breed Ids for the Dog API (will allow requesting images)

In [3]:
load_dotenv()
dog_api_key = os.getenv("DOG_API_KEY")

import requests

results = requests.get(f"https://api.thedogapi.com/v1/breeds")
results = results.json()
df_dog_data = [{'Name': x['name'].lower(), 'Dog_api_id': x['id']} for x in results]
df_dog_data = pd.DataFrame(df_dog_data)
df_dog_data.head()

,Name,Dog_api_id
0,affenpinscher,1
1,afghan hound,2
2,african hunting dog,3
3,airedale terrier,4
4,akbash dog,5


Merge Kaggle dataset with Dog API data

In [4]:
df_merged = pd.merge(df, df_dog_data, on='Name', how='inner')
df_merged.head(10)

,Name,min_life_expectancy,max_life_expectancy,max_height_male,max_height_female,max_weight_male,max_weight_female,min_height_male,min_height_female,min_weight_male,...,grooming,drooling,coat_length,good_with_strangers,playfulness,protectiveness,trainability,energy,barking,Dog_api_id
0,golden retriever,10,12,24.0,24.0,75.0,65.0,23.0,23.0,65.0,...,2,2,1,5,4,3,5,3,1,121
1,labrador retriever,10,12,24.5,24.5,80.0,70.0,22.5,22.5,65.0,...,2,2,1,5,5,3,5,5,3,149
2,great dane,7,10,32.0,32.0,175.0,140.0,30.0,30.0,140.0,...,1,4,1,3,4,5,3,4,3,124
3,boxer,10,12,25.0,25.0,80.0,65.0,23.0,23.0,65.0,...,2,3,1,4,4,4,4,4,3,55
4,dalmatian,11,13,24.0,24.0,70.0,70.0,19.0,19.0,45.0,...,2,2,1,4,4,4,4,4,3,92
5,poodle (miniature),10,18,15.0,15.0,15.0,15.0,10.0,10.0,10.0,...,4,1,1,5,5,3,5,4,4,196
6,bullmastiff,7,9,27.0,27.0,130.0,120.0,25.0,25.0,110.0,...,1,3,1,3,3,5,4,4,1,64
7,beagle,10,15,16.0,15.0,20.0,30.0,14.0,13.0,15.0,...,2,1,1,3,4,2,3,4,4,31
8,border collie,12,15,22.0,22.0,55.0,55.0,19.0,19.0,30.0,...,3,1,1,4,5,3,5,5,4,50
9,siberian husky,12,14,23.5,23.5,60.0,50.0,21.0,21.0,45.0,...,2,1,1,5,5,1,3,5,5,226


Load akc dataset

In [5]:
df_akc = pd.read_csv('../data/akc-data-latest.csv')
df_akc.rename(columns={'Unnamed: 0': 'Name'}, inplace=True)
df_akc['Name'] = df_akc['Name'].str.lower()
df_akc.head()

,Name,description,temperament,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,...,grooming_frequency_value,grooming_frequency_category,shedding_value,shedding_category,energy_level_value,energy_level_category,trainability_value,trainability_category,demeanor_value,demeanor_category
0,affenpinscher,The Affen’s apish look has been described many...,"Confident, Famously Funny, Fearless",148,22.86,29.21,3.175147,4.535924,12.0,15.0,...,0.6,2-3 Times a Week Brushing,0.6,Seasonal,0.6,Regular Exercise,0.8,Easy Training,1.0,Outgoing
1,afghan hound,"The Afghan Hound is an ancient breed, his whol...","Dignified, Profoundly Loyal, Aristocratic",113,63.50,68.58,22.679619,27.215542,12.0,15.0,...,0.8,Daily Brushing,0.2,Infrequent,0.8,Energetic,0.2,May be Stubborn,0.2,Aloof/Wary
2,airedale terrier,The Airedale Terrier is the largest of all ter...,"Friendly, Clever, Courageous",60,58.42,58.42,22.679619,31.751466,11.0,14.0,...,0.6,2-3 Times a Week Brushing,0.4,Occasional,0.6,Regular Exercise,1.0,Eager to Please,0.8,Friendly
3,akita,"Akitas are burly, heavy-boned spitz-type dogs ...","Courageous, Dignified, Profoundly Loyal",47,60.96,71.12,31.751466,58.967008,10.0,13.0,...,0.8,Daily Brushing,0.6,Seasonal,0.8,Energetic,1.0,Eager to Please,0.6,Alert/Responsive
4,alaskan malamute,The Alaskan Malamute stands 23 to 25 inches at...,"Affectionate, Loyal, Playful",58,58.42,63.50,34.019428,38.555351,10.0,14.0,...,0.6,2-3 Times a Week Brushing,0.6,Seasonal,0.8,Energetic,0.4,Independent,0.8,Friendly


Merge with other data

In [6]:
df_merged2 = pd.merge(df_merged, df_akc[[
    'Name', 'description', 'popularity', 'temperament', 'grooming_frequency_category', 'shedding_category', 
    'energy_level_category', 'trainability_category', 'demeanor_category', 'min_height', 'max_height',
    'min_weight', 'max_weight', 'max_expectancy'
]], on='Name', how='inner')
df_merged2.head()

,Name,min_life_expectancy,max_life_expectancy,max_height_male,max_height_female,max_weight_male,max_weight_female,min_height_male,min_height_female,min_weight_male,...,grooming_frequency_category,shedding_category,energy_level_category,trainability_category,demeanor_category,min_height,max_height,min_weight,max_weight,max_expectancy
0,golden retriever,10,12,24.0,24.0,75.0,65.0,23.0,23.0,65.0,...,Weekly Brushing,Seasonal,Needs Lots of Activity,Eager to Please,Friendly,54.61,60.96,24.947580,34.019428,12.0
1,labrador retriever,10,12,24.5,24.5,80.0,70.0,22.5,22.5,65.0,...,Weekly Brushing,Regularly,Needs Lots of Activity,Eager to Please,Outgoing,54.61,62.23,24.947580,36.287390,12.0
2,great dane,7,10,32.0,32.0,175.0,140.0,30.0,30.0,140.0,...,Occasional Bath/Brush,Seasonal,Energetic,Agreeable,Alert/Responsive,71.12,81.28,49.895161,79.378665,10.0
3,boxer,10,12,25.0,25.0,80.0,65.0,23.0,23.0,65.0,...,Weekly Brushing,Occasional,Energetic,Easy Training,Outgoing,54.61,63.50,6.803886,36.287390,12.0
4,dalmatian,11,13,24.0,24.0,70.0,70.0,19.0,19.0,45.0,...,Weekly Brushing,Frequent,Energetic,Agreeable,Alert/Responsive,48.26,60.96,20.411657,31.751466,13.0


Drop some unused columns from the merged data

In [7]:
df_merged2.drop(columns=[
    'shedding', 'grooming', 'drooling', 'coat_length', 'trainability', 
    'energy', 'min_life_expectancy', 'max_height_male',
    'max_height_female', 'max_weight_male', 'max_weight_female',
    'min_height_male', 'min_height_female', 'min_weight_male',
    'min_weight_female', 'max_expectancy', 'playfulness', 'protectiveness', 'barking'
], inplace=True)
df_merged2.head()

,Name,max_life_expectancy,good_with_children,good_with_other_dogs,good_with_strangers,Dog_api_id,description,popularity,temperament,grooming_frequency_category,shedding_category,energy_level_category,trainability_category,demeanor_category,min_height,max_height,min_weight,max_weight
0,golden retriever,12,5,5,5,121,"The Golden Retriever is a sturdy, muscular dog...",3,"Friendly, Intelligent, Devoted",Weekly Brushing,Seasonal,Needs Lots of Activity,Eager to Please,Friendly,54.61,60.96,24.947580,34.019428
1,labrador retriever,12,5,5,5,149,"The sturdy, well-balanced Labrador Retriever c...",1,"Friendly, Active, Outgoing",Weekly Brushing,Regularly,Needs Lots of Activity,Eager to Please,Outgoing,54.61,62.23,24.947580,36.287390
2,great dane,10,3,3,3,124,"As tall as 32 inches at the shoulder, Danes to...",16,"Friendly, Patient, Dependable",Occasional Bath/Brush,Seasonal,Energetic,Agreeable,Alert/Responsive,71.12,81.28,49.895161,79.378665
3,boxer,12,5,3,4,55,A well-made Boxer in peak condition is an awes...,11,"Bright, Fun-Loving, Active",Weekly Brushing,Occasional,Energetic,Easy Training,Outgoing,54.61,63.50,6.803886,36.287390
4,dalmatian,13,3,3,4,92,"The Dalmatian’s delightful, eye-catching spots...",56,"Dignified, Smart, Outgoing",Weekly Brushing,Frequent,Energetic,Agreeable,Alert/Responsive,48.26,60.96,20.411657,31.751466


Create some aggregated features

In [8]:
# average height
df_merged2['avg_height'] = df_merged2[['min_height', 'max_height']].mean(axis=1)

# bin the size into 3 text categories
df_merged2['size'] = pd.cut(df_merged2['avg_height'], bins=3, labels=['small', 'medium', 'large'])
df_merged2.drop(columns=['min_height', 'max_height', 'min_weight', 'max_weight', 'avg_height'], inplace=True)

# bin popularity into 3 text categories from rare to popular
df_merged2['popularity'] = pd.to_numeric(df_merged2['popularity'], errors='coerce')
df_merged2['popularity'].astype('float64')
df_merged2['popularity'].fillna(df_merged2['popularity'].median(), inplace=True)
df_merged2['popularity'] = pd.cut(df_merged2['popularity'], bins=3, labels=['rare', 'average', 'popular'])

# make friendliness from good_with_children, good_with_dogs, good_with_strangers to 3 categories
df_merged2['friendliness'] = df_merged2[['good_with_children', 'good_with_other_dogs', 'good_with_strangers']].mean(axis=1)
df_merged2['friendliness'] = pd.cut(df_merged2['friendliness'], bins=3, labels=['low', 'medium', 'high'])
df_merged2.drop(columns=['good_with_children', 'good_with_other_dogs', 'good_with_strangers'], inplace=True)

for col in df_merged2.select_dtypes(include=['object']):
    df_merged2[col] = df_merged2[col].fillna("unknown")
df_merged2.head()


C:\Users\karen\AppData\Local\Temp\ipykernel_3528\541214414.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged2['popularity'].fillna(df_merged2['popularity'].median(), inplace=True)


,Name,max_life_expectancy,Dog_api_id,description,popularity,temperament,grooming_frequency_category,shedding_category,energy_level_category,trainability_category,demeanor_category,size,friendliness
0,golden retriever,12,121,"The Golden Retriever is a sturdy, muscular dog...",rare,"Friendly, Intelligent, Devoted",Weekly Brushing,Seasonal,Needs Lots of Activity,Eager to Please,Friendly,large,high
1,labrador retriever,12,149,"The sturdy, well-balanced Labrador Retriever c...",rare,"Friendly, Active, Outgoing",Weekly Brushing,Regularly,Needs Lots of Activity,Eager to Please,Outgoing,large,high
2,great dane,10,124,"As tall as 32 inches at the shoulder, Danes to...",rare,"Friendly, Patient, Dependable",Occasional Bath/Brush,Seasonal,Energetic,Agreeable,Alert/Responsive,large,low
3,boxer,12,55,A well-made Boxer in peak condition is an awes...,rare,"Bright, Fun-Loving, Active",Weekly Brushing,Occasional,Energetic,Easy Training,Outgoing,large,medium
4,dalmatian,13,92,"The Dalmatian’s delightful, eye-catching spots...",rare,"Dignified, Smart, Outgoing",Weekly Brushing,Frequent,Energetic,Agreeable,Alert/Responsive,medium,medium


Split into selectors and ddescriptive info and export as CSV

In [9]:
df_select = df_merged2[['size', 'popularity', 'energy_level_category', 'trainability_category', 'grooming_frequency_category', 
                        'shedding_category', 'demeanor_category', 'friendliness']]
df_description = df_merged2[['Name', 'description', 'temperament', 'max_life_expectancy', 'Dog_api_id']]

In [10]:
df_select.to_csv("../data/dog_breeds_select.csv", index=False)
df_description.to_csv("../data/dog_breeds_description.csv", index=False)